# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8effb6fa00>
├── 'a' --> tensor([[ 1.3329,  0.8161,  0.4443],
│                   [-0.4086,  1.7039,  1.3857]])
└── 'x' --> <FastTreeValue 0x7f8effb75340>
    └── 'c' --> tensor([[-0.2031, -1.3267, -0.2861,  0.5325],
                        [-0.5568,  0.6618,  2.6936, -0.3328],
                        [ 1.1556, -1.7019, -0.8625,  0.3515]])

In [4]:
t.a

tensor([[ 1.3329,  0.8161,  0.4443],
        [-0.4086,  1.7039,  1.3857]])

In [5]:
%timeit t.a

61.3 ns ± 0.0702 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8effb6fa00>
├── 'a' --> tensor([[ 1.2933, -0.1150, -1.1617],
│                   [ 0.9862,  0.9825,  1.4717]])
└── 'x' --> <FastTreeValue 0x7f8effb75340>
    └── 'c' --> tensor([[-0.2031, -1.3267, -0.2861,  0.5325],
                        [-0.5568,  0.6618,  2.6936, -0.3328],
                        [ 1.1556, -1.7019, -0.8625,  0.3515]])

In [7]:
%timeit t.a = new_value

61.3 ns ± 0.0282 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.3329,  0.8161,  0.4443],
               [-0.4086,  1.7039,  1.3857]]),
    x: Batch(
           c: tensor([[-0.2031, -1.3267, -0.2861,  0.5325],
                      [-0.5568,  0.6618,  2.6936, -0.3328],
                      [ 1.1556, -1.7019, -0.8625,  0.3515]]),
       ),
)

In [10]:
b.a

tensor([[ 1.3329,  0.8161,  0.4443],
        [-0.4086,  1.7039,  1.3857]])

In [11]:
%timeit b.a

58.6 ns ± 0.019 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.1765, -0.3319, -0.8142],
               [-1.4743, -0.2655,  0.1244]]),
    x: Batch(
           c: tensor([[-0.2031, -1.3267, -0.2861,  0.5325],
                      [-0.5568,  0.6618,  2.6936, -0.3328],
                      [ 1.1556, -1.7019, -0.8625,  0.3515]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.0627 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

840 ns ± 0.327 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 31.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 6.26 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 187 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8e507b22b0>
├── 'a' --> tensor([[[ 1.3329,  0.8161,  0.4443],
│                    [-0.4086,  1.7039,  1.3857]],
│           
│                   [[ 1.3329,  0.8161,  0.4443],
│                    [-0.4086,  1.7039,  1.3857]],
│           
│                   [[ 1.3329,  0.8161,  0.4443],
│                    [-0.4086,  1.7039,  1.3857]],
│           
│                   [[ 1.3329,  0.8161,  0.4443],
│                    [-0.4086,  1.7039,  1.3857]],
│           
│                   [[ 1.3329,  0.8161,  0.4443],
│                    [-0.4086,  1.7039,  1.3857]],
│           
│                   [[ 1.3329,  0.8161,  0.4443],
│                    [-0.4086,  1.7039,  1.3857]],
│           
│                   [[ 1.3329,  0.8161,  0.4443],
│                    [-0.4086,  1.7039,  1.3857]],
│           
│                   [[ 1.3329,  0.8161,  0.4443],
│                    [-0.4086,  1.7039,  1.3857]]])
└── 'x' --> <FastTreeValue 0x7f8e507b2b50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 60 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8e507d3f40>
├── 'a' --> tensor([[ 1.3329,  0.8161,  0.4443],
│                   [-0.4086,  1.7039,  1.3857],
│                   [ 1.3329,  0.8161,  0.4443],
│                   [-0.4086,  1.7039,  1.3857],
│                   [ 1.3329,  0.8161,  0.4443],
│                   [-0.4086,  1.7039,  1.3857],
│                   [ 1.3329,  0.8161,  0.4443],
│                   [-0.4086,  1.7039,  1.3857],
│                   [ 1.3329,  0.8161,  0.4443],
│                   [-0.4086,  1.7039,  1.3857],
│                   [ 1.3329,  0.8161,  0.4443],
│                   [-0.4086,  1.7039,  1.3857],
│                   [ 1.3329,  0.8161,  0.4443],
│                   [-0.4086,  1.7039,  1.3857],
│                   [ 1.3329,  0.8161,  0.4443],
│                   [-0.4086,  1.7039,  1.3857]])
└── 'x' --> <FastTreeValue 0x7f8ed42ddee0>
    └── 'c' --> tensor([[-0.2031, -1.3267, -0.2861,  0.5325],
                        [-0.5568,  0.6618,  2.6936, -0.3328],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 68.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.7 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.2031, -1.3267, -0.2861,  0.5325],
                       [-0.5568,  0.6618,  2.6936, -0.3328],
                       [ 1.1556, -1.7019, -0.8625,  0.3515]],
              
                      [[-0.2031, -1.3267, -0.2861,  0.5325],
                       [-0.5568,  0.6618,  2.6936, -0.3328],
                       [ 1.1556, -1.7019, -0.8625,  0.3515]],
              
                      [[-0.2031, -1.3267, -0.2861,  0.5325],
                       [-0.5568,  0.6618,  2.6936, -0.3328],
                       [ 1.1556, -1.7019, -0.8625,  0.3515]],
              
                      [[-0.2031, -1.3267, -0.2861,  0.5325],
                       [-0.5568,  0.6618,  2.6936, -0.3328],
                       [ 1.1556, -1.7019, -0.8625,  0.3515]],
              
                      [[-0.2031, -1.3267, -0.2861,  0.5325],
                       [-0.5568,  0.6618,  2.6936, -0.3328],
                       [ 1.1556, -1.7019, -0.8625,  0.3515]],

In [26]:
%timeit Batch.stack(batches)

78 µs ± 188 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.2031, -1.3267, -0.2861,  0.5325],
                      [-0.5568,  0.6618,  2.6936, -0.3328],
                      [ 1.1556, -1.7019, -0.8625,  0.3515],
                      [-0.2031, -1.3267, -0.2861,  0.5325],
                      [-0.5568,  0.6618,  2.6936, -0.3328],
                      [ 1.1556, -1.7019, -0.8625,  0.3515],
                      [-0.2031, -1.3267, -0.2861,  0.5325],
                      [-0.5568,  0.6618,  2.6936, -0.3328],
                      [ 1.1556, -1.7019, -0.8625,  0.3515],
                      [-0.2031, -1.3267, -0.2861,  0.5325],
                      [-0.5568,  0.6618,  2.6936, -0.3328],
                      [ 1.1556, -1.7019, -0.8625,  0.3515],
                      [-0.2031, -1.3267, -0.2861,  0.5325],
                      [-0.5568,  0.6618,  2.6936, -0.3328],
                      [ 1.1556, -1.7019, -0.8625,  0.3515],
                      [-0.2031, -1.3267, -0.2861,  0.5325],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 309 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

313 µs ± 801 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
